<a href="https://colab.research.google.com/github/oonid/growth-hacking-with-nlp-sentiment-analysis/blob/master/create_neural_network_based_sentiment_analyzers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network Based Sentiment Analyzers

### requirements setup

includes: 

*   transformers
*   simpletransformers
*   tokenizers



In [1]:
!pip install --upgrade transformers  # make sure compatible with tokenizers
!wget https://raw.githubusercontent.com/crow-intelligence/growth-hacking-sentiment/master/requirements.txt
!pip install -r requirements.txt

     |████████████████████████████████| 675kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 10.0MB/s 
     |████████████████████████████████| 3.8MB 19.4MB/s 
     |████████████████████████████████| 890kB 39.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=68ba5a6754b9ea25de82abd07cf76fccef33818534e32524a1ed7cf72ad87318
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
--2020-06-27 09:38:36--  https://raw.githubusercontent.com/crow-intelligence/growth-hacking-sentiment/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[=====

### Install apex

make sure to switch Google Colab runtime type to GPU hardware accelerator


In [2]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [3]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 7293 (delta 20), reused 19 (delta 6), pack-reused 7255
Receiving objects: 100% (7293/7293), 13.87 MiB | 26.40 MiB/s, done.
Resolving deltas: 100% (4919/4919), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-jhbet3oz
Created temporary directory: /tmp/pip-req-tracker-ho6_0kjo
Created requirements tracker '/tmp/pip-req-tracker-ho6_0kjo'
Created temporary directory: /tmp/pip-install-klunwt1v
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-hmehv1pu
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-ho6_0kjo'
    Running setup.py (path:

In [4]:
# connect google colab to google driver if needed (uncomment 2 lines below)
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [5]:
# all imports and its related

%matplotlib inline

import pandas as pd
import numpy as np

from transformers import pipeline, BertModel, BertTokenizer
from simpletransformers.classification import ClassificationModel

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


### load the small_corpus CSV

run process from [create_dataset.ipynb](https://github.com/oonid/growth-hacking-with-nlp-sentiment-analysis/blob/master/create_dataset.ipynb)

copy file **small_corpus.csv** to this Google Colab Files (via file upload or mount drive)


In [6]:
df = pd.read_csv('small_corpus.csv')
df

,ratings,reviews
0,1,Recently UBISOFT had to settle a huge class-ac...
1,1,"code didn't work, got me a refund."
2,1,"these do not work at all, all i get is static ..."
3,1,well let me start by saying that when i first ...
4,1,"Dont waste your money, you will just end up us..."
...,...,...
4495,5,"Nice long micro USB cable, battery lasts a lon..."
4496,5,I've been having a great time with this game. ...
4497,5,d
4498,5,"Really pretty, funny, interesting game. Works ..."


In [7]:
# check if any columns has null, and yes the reviews column has
df.isnull().any()

ratings    False
reviews     True
dtype: bool

In [8]:
# repair null in column reviews with empty string ''
df.reviews = df.reviews.fillna('')

# test again
df.isnull().any()

ratings    False
reviews    False
dtype: bool

In [9]:
# Setting up transformers pipeline and tokenizer
sentiment_analysis_pipeline = pipeline('sentiment-analysis')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


In [10]:
for row in pd.concat([df.head(2), df.tail(2)]).itertuples(index=False):
    # print(row.reviews)
    review = row.reviews
    if len(review) > 512: # chop to 512, maximum sequence length for this model
        review = review[:512]
        print('[warning] review get truncated.')
        # TODO find alternative to handle long sentences,
        # TODO shall we tokenize to multiple sentences and process per sentence?
    e = tokenizer.encode(review)
    d = tokenizer.decode(e, skip_special_tokens=True)
    print(d)
    s = sentiment_analysis_pipeline(d)
    print(s)
    print('\n--\n')


[warning] review get truncated.
recently ubisoft had to settle a huge class - action suit brought against the company for bundling ( the notoriously harmful ) starforce drm with its released games. so what the geniuses at the helm do next? they decide to make the same mistake yet again - by choosing the same drm scheme that made bioshock, mass effect and spore infamous : securom 7. xx with limited activations! mass effect can be found in clearance bins only months after its release ; spore not only undersold miserably but also made history as t
[{'label': 'NEGATIVE', 'score': 0.9991271495819092}]

--

code didn't work, got me a refund.
[{'label': 'NEGATIVE', 'score': 0.9996480345726013}]

--

really pretty, funny, interesting game. works well. i recommend it for all ages.
[{'label': 'POSITIVE', 'score': 0.9998774528503418}]

--

i had a lot of fun playing this game, if your looking for a game to jump into with not much overthinking this is the game
[{'label': 'POSITIVE', 'score': 0.987

In [11]:
def score_review(review):
    if len(review) > 512: # chop to 512, maximum sequence length for this model
        # print('\n[warning] review get chopped.\n{}'.format(review))
        review = review[:512]
        # TODO find alternative to handle long sentences,
        # TODO shall we tokenize to multiple sentences and process per sentence?
    e = tokenizer.encode(review)
    d = tokenizer.decode(e, skip_special_tokens=True)
    s = sentiment_analysis_pipeline(d)
    if len(s) != 1:
        raise ValueError('got multiple sentiment analysis results.')

    # s[0] the first and only sentiment analysis result
    if s[0]['label'] == 'POSITIVE' and s[0]['score'] > 0.85:
        return 1
    elif s[0]['label'] == 'NEGATIVE' and s[0]['score'] > 0.95:
        return -1
    else:  # NEUTRAL
        return 0

sentiment_classes = df['reviews'].apply(score_review)
sentiment_classes

0      -1
1      -1
2      -1
3      -1
4      -1
       ..
4495   -1
4496    1
4497    1
4498    1
4499    1
Name: reviews, Length: 4500, dtype: int64

In [12]:
def categorize_rating(rating):
    if rating == 5:
        return 1  # 'positive'
    elif 2 <= rating <= 4:
        return 0  # 'neutral'
    else:  # rating == 1
        return -1  # 'negative'

rating_classes = df['ratings'].apply(categorize_rating)
rating_classes

0      -1
1      -1
2      -1
3      -1
4      -1
       ..
4495    1
4496    1
4497    1
4498    1
4499    1
Name: ratings, Length: 4500, dtype: int64

In [13]:
df['sentiment classes'] = sentiment_classes
df['rating classes'] = rating_classes
df

,ratings,reviews,sentiment classes,rating classes
0,1,Recently UBISOFT had to settle a huge class-ac...,-1,-1
1,1,"code didn't work, got me a refund.",-1,-1
2,1,"these do not work at all, all i get is static ...",-1,-1
3,1,well let me start by saying that when i first ...,-1,-1
4,1,"Dont waste your money, you will just end up us...",-1,-1
...,...,...,...,...
4495,5,"Nice long micro USB cable, battery lasts a lon...",-1,1
4496,5,I've been having a great time with this game. ...,1,1
4497,5,d,1,1
4498,5,"Really pretty, funny, interesting game. Works ...",1,1


In [14]:
y_pred = list(df['sentiment classes'])
y_true = list(df['rating classes'])

print('accuracy score: {}'.format(accuracy_score(y_true=y_true, y_pred=y_pred)))
print('accuracy score (normalized): {}'\
      .format(accuracy_score(y_true=y_true, y_pred=y_pred, normalize=True)))

accuracy score: 0.6106666666666667
accuracy score (normalized): 0.6106666666666667


In [15]:

target_names = ['class Negative', 'class Neutral', 'class Positive']

print('classification report:\n{}'\
      .format(classification_report(y_true=y_true, y_pred=y_pred,
                                    target_names=target_names)))

classification report:
                precision    recall  f1-score   support

class Negative       0.59      0.89      0.71      1500
 class Neutral       0.43      0.08      0.14      1500
class Positive       0.67      0.86      0.75      1500

      accuracy                           0.61      4500
     macro avg       0.56      0.61      0.53      4500
  weighted avg       0.56      0.61      0.53      4500



# Create a training set and a test set

Create a training set and a test set out of the reviews to create a neural network based our on settings.

In [16]:
def encode_labels(label):
    """encode [-1, 0, 1] to [0, 1, 2]"""
    if label == -1:
        return 0  # negative
    elif label == 0:
        return 1  # neutral
    elif label == 1:
        return 2  # positive
    else:
        raise ValueError('unknown label value')
def decode_labels(label):
    """decode [0, 1, 2] to [-1, 0, 1]"""
    if label == 0:  # negative
        return -1
    elif label == 1:  # neutral
        return 0
    elif label == 2:  # positive
        return 1
    else:
        raise ValueError('unknown label value')

encoded_labels = df['rating classes'].apply(encode_labels)
encoded_labels.unique()

array([0, 1, 2])

In [17]:
X = list(df['reviews'])
y = list(encoded_labels)  # positive, neutral, negative

# default test_size = 0.25

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,
                                                    test_size=0.25)
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))


3375
3375
1125
1125


In [18]:
train_df = pd.DataFrame({
    'text': X_train,
    'labels': y_train,
})
train_df

,text,labels
0,The PS3 felt like PS vita coming out the box. ...,2
1,Waste of money. Nothing new.,0
2,When Need for Speed Hot Persuit 2 failed to de...,1
3,Buggy hardly any servers available. Always ask...,0
4,Cool concept but ultimately feels uninspired a...,0
...,...,...
3370,too difficult and not enough hints to even be ...,0
3371,I have had this game since June 5th and have h...,1
3372,"It works fine, the game itself is like a sweat...",1
3373,Great game system. This is perfect for my 7 ye...,2


In [19]:
eval_df = pd.DataFrame({
    'text': X_test,
    'labels': y_test,
})
eval_df

,text,labels
0,"Good headset nice mic, picks up a lot of back...",1
1,Fun game. Cool little side story from the main...,2
2,"OK, so I finished Dantes Inferno and it kicked...",1
3,"all excellent , thank you very much",2
4,I have played plenty of rpg and plenty of beth...,0
...,...,...
1120,Best PC controller currently available. So muc...,2
1121,To start: Left 4 Dead 2 is not a sequel but th...,1
1122,If you like games with Amazing graphics and pr...,0
1123,Basically... This game sucks. The graphics suc...,0


# Create our own neural network-based sentiment classifier

### Create model

In [20]:
# Create a ClassificationModel
model = ClassificationModel(model_type='distilbert',
                            model_name='distilbert-base-uncased',
                            num_labels=3,
                            args={
                              'use_cuda': True, 
                              'max_seq_length': 128,
                              'num_train_epochs': 10,
                              'output_dir': 'model_dir/',
                              'best_model_dir': 'model_dir/',
                              'evaluate_during_training': True,
                              'num_training_epochs': 20,
                              'train_batch_size': 20,
                              'eval_batch_size': 20
                            })

### Training

also evaluate during training (see argument `evaluate_during_training` set as True).

In [21]:
# Train the model
model.train_model(train_df=train_df, eval_df=eval_df)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.137975

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 1.074314

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

Running loss: 0.002367


Running loss: 0.012032


Running loss: 0.014835


Running loss: 0.001551


Running loss: 0.000768



In [22]:
result, model_outputs, wrong_predictions = \
    model.eval_model(eval_df, acc=accuracy_score)
print(result)



{'mcc': 0.4799381831618073, 'acc': 0.6515555555555556, 'eval_loss': 1.8006001091848143}


In [23]:
y_pred = list(model_outputs.argmax(axis=1))
y_true = y_test

print('accuracy score: {}'.format(accuracy_score(y_true=y_true, y_pred=y_pred)))
print('accuracy score (normalized): {}'\
      .format(accuracy_score(y_true=y_true, y_pred=y_pred, normalize=True)))

target_names = ['class Negative', 'class Neutral', 'class Positive']

print('classification report:\n{}'\
      .format(classification_report(y_true=y_true, y_pred=y_pred,
                                    target_names=target_names)))

accuracy score: 0.6515555555555556
accuracy score (normalized): 0.6515555555555556
classification report:
                precision    recall  f1-score   support

class Negative       0.70      0.63      0.66       375
 class Neutral       0.53      0.63      0.57       375
class Positive       0.77      0.69      0.73       375

      accuracy                           0.65      1125
     macro avg       0.67      0.65      0.66      1125
  weighted avg       0.67      0.65      0.66      1125



# Experiment with different parameter settings

**ATTN for Colab User with GPU**

before continue, please stop few minutes,
(after training above model).
if you continue the training directly, the GPU memory will not enough.

### Create model

In [24]:
# Create a ClassificationModel
model512 = ClassificationModel(model_type='distilbert',
                               model_name='distilbert-base-uncased',
                               num_labels=3,
                               args={
                                   'use_cuda': True, 
                                   'max_seq_length': 512,
                                   'sliding_window': True,
                                   'num_train_epochs': 10,
                                   'output_dir': 'model512_dir/',
                                   'best_model_dir': 'model512_dir/',
                                   'evaluate_during_training': True,
                                   'num_training_epochs': 20,
                                   'train_batch_size': 20,
                                   'eval_batch_size': 20,
                                })

### Training

also evaluate during training (see argument `evaluate_during_training` set as True).

In [25]:
# Train the model
model512.train_model(train_df=train_df, eval_df=eval_df)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.112055

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 1.023448

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

Running loss: 0.001328


Running loss: 0.000496


Running loss: 0.003133


Running loss: 0.001142


Running loss: 0.000359



In [26]:
result, model_outputs, wrong_predictions = \
    model512.eval_model(eval_df, acc=accuracy_score)
print(result)
if isinstance(model_outputs, list):
    flat_outputs = []
    for o in model_outputs:
        if o.shape != (1, 3):
            # get mean value from multiple results
            oo = np.sum(o, axis=0)
            oo = np.reshape(oo, (-1, 3))
            flat_outputs.append(oo)
        else:
            flat_outputs.append(o)
    # overwrite with flat outputs, get first index because after stack (1, 1125, 3)
    model_outputs = np.stack(flat_outputs, axis=1)[0]
    print(model_outputs.shape)


{'mcc': 0.502159488736401, 'acc': 0.6675555555555556, 'eval_loss': 1.9439644136212089}
(1125, 3)


In [27]:

y_pred = list(model_outputs.argmax(axis=1))
y_true = y_test

print('accuracy score: {}'.format(accuracy_score(y_true=y_true, y_pred=y_pred)))
print('accuracy score (normalized): {}'\
      .format(accuracy_score(y_true=y_true, y_pred=y_pred, normalize=True)))

target_names = ['class Negative', 'class Neutral', 'class Positive']

print('classification report:\n{}'\
      .format(classification_report(y_true=y_true, y_pred=y_pred,
                                    target_names=target_names)))

accuracy score: 0.6737777777777778
accuracy score (normalized): 0.6737777777777778
classification report:
                precision    recall  f1-score   support

class Negative       0.71      0.67      0.69       375
 class Neutral       0.54      0.61      0.58       375
class Positive       0.79      0.73      0.76       375

      accuracy                           0.67      1125
     macro avg       0.68      0.67      0.68      1125
  weighted avg       0.68      0.67      0.68      1125

